<a href="https://colab.research.google.com/github/yomyaykya/yomyay/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from keras.utils.np_utils import to_categorical
from keras import regularizers
import numpy as np

In [2]:
def extract_label(img_path, train=True):
    filename, _ = os.path.splitext(os.path.basename(img_path))
    subject_id, etc = filename.split('__')
    
    if train:
        gender, lr, finger, _, _ = etc.split('_')
    else:
        gender, lr, finger, _ = etc.split('_')
    
    gender = 0 if gender == 'M' else 1
    lr =0 if lr == 'Left' else 1
    
    if finger == 'thumb':
        finger = 0
    elif finger == 'index':
        finger = 1
     
    return np.array([finger], dtype=np.uint16)

In [3]:
img_size = 96
def loading_data(path,train):
    print("loading data from: ",path)
    data = []
    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            img_resize = cv2.resize(img_array, (img_size, img_size))
            label = extract_label(os.path.join(path, img),train)
            data.append([label[0], img_resize ])
        except Exception as e:
            pass
    data
    return data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
Real_path = "/content/drive/MyDrive/For thesis/SOCOFing/Real"
Easy_path = "/content/drive/MyDrive/For thesis/SOCOFing/Altered/Altered-Easy"
Medium_path = "/content/drive/MyDrive/For thesis/SOCOFing/Altered/Altered-Medium"
Hard_path = "/content/drive/MyDrive/For thesis/SOCOFing/Altered/Altered-Hard"


Easy_data = loading_data(Easy_path, train = True)
Medium_data = loading_data(Medium_path, train = True)
Hard_data = loading_data(Hard_path, train = True)
test = loading_data(Real_path, train = False)

data = np.concatenate([Easy_data, Medium_data, Hard_data], axis=0)

del Easy_data, Medium_data, Hard_data

loading data from:  /content/drive/MyDrive/For thesis/SOCOFing/Altered/Altered-Easy
loading data from:  /content/drive/MyDrive/For thesis/SOCOFing/Altered/Altered-Medium
loading data from:  /content/drive/MyDrive/For thesis/SOCOFing/Altered/Altered-Hard
loading data from:  /content/drive/MyDrive/For thesis/SOCOFing/Real


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [6]:
import random
random.shuffle(data)
random.shuffle(test)

In [7]:
data

array([[1, array([[160, 158, 158, ...,   0,   0,   0],
                  [160, 105, 121, ...,   0,   0,   0],
                  [160, 105, 255, ...,   0,   0,   0],
                  ...,
                  [  0,   0,   0, ...,   0,   0,   0],
                  [  0,   0,   0, ...,   0,   0,   0],
                  [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8)],
       [1, array([[160, 158, 158, ...,   0,   0,   0],
                  [160, 105, 121, ...,   0,   0,   0],
                  [160, 105, 255, ...,   0,   0,   0],
                  ...,
                  [  0,   0,   0, ...,   0,   0,   0],
                  [  0,   0,   0, ...,   0,   0,   0],
                  [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8)],
       [0, array([[160, 158, 158, ...,   0,   0,   0],
                  [160, 105, 121, ...,   0,   0,   0],
                  [160, 105, 255, ...,   0,   0,   0],
                  ...,
                  [  0,   0,   0, ...,   0,   0,   0],
                  [

In [8]:
img, labels = [], []
for label, feature in data:
    labels.append(label)
    img.append(feature)
train_data = np.array(img).reshape(-1, img_size, img_size, 1)
train_data = train_data / 255.0
train_labels = to_categorical(labels, num_classes = 2)

In [14]:
#Designing compiling the CNN model 
model=Sequential() 
model.add(Conv2D(input_shape= (96,96,1),filters=64,kernel_size=(3,3),padding='same',activation='relu')) 
model.add(Conv2D(filters=64,kernel_size=(3,3),padding='same',activation='relu')) 
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128,kernel_size=(3,3),padding='same',activation='relu')) 
model.add(Conv2D(filters=128,kernel_size=(3,3),padding='same',activation='relu')) 
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=256,kernel_size=(3,3),padding='same',activation='relu'))
model.add(Conv2D(filters=256,kernel_size=(3,3),padding='same',activation='relu')) 
model.add(Conv2D(filters=256,kernel_size=(3,3),padding='same',activation='relu')) 
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=512,kernel_size=(3,3),padding='same',activation='relu')) 
model.add(Conv2D(filters=512,kernel_size=(3,3),padding='same',activation='relu')) 
model.add(Conv2D(filters=512,kernel_size=(3,3),padding='same',activation='relu')) 
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten()) 
model.add(Dense(4096,activation='relu')) 
model.add(Dense(4096,activation='relu')) 
model.add(Dense(2,activation='softmax')) 

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy']) 

model.summary() 

from keras.callbacks import ModelCheckpoint,EarlyStopping 
checkpoint=ModelCheckpoint("vgg16_1.h5",monitor='val_acc',verbose=1,save_best_only=True) 
early=EarlyStopping(monitor='val_acc',patience=20,verbose=1)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_30 (Conv2D)          (None, 96, 96, 64)        640       
                                                                 
 conv2d_31 (Conv2D)          (None, 96, 96, 64)        36928     
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 48, 48, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_32 (Conv2D)          (None, 48, 48, 128)       73856     
                                                                 
 conv2d_33 (Conv2D)          (None, 48, 48, 128)       147584    
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 24, 24, 128)      0         
 g2D)                                                 

In [16]:
model.save("fingerprint.h5")

In [15]:
model.compile(optimizer = optimizers.Adam(learning_rate=0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(train_data, train_labels, batch_size = 128, epochs = 10,validation_split = 0.2
          )

Epoch 1/10
21/21 [==============================] - 1054s 50s/step - loss: 0.6063 - accuracy: 0.6554 - val_loss: 0.4249 - val_accuracy: 0.7946
Epoch 2/10
21/21 [==============================] - 1042s 50s/step - loss: 0.3392 - accuracy: 0.8571 - val_loss: 0.2515 - val_accuracy: 0.9100
Epoch 3/10
21/21 [==============================] - 1062s 51s/step - loss: 0.2385 - accuracy: 0.9003 - val_loss: 0.2519 - val_accuracy: 0.8951
Epoch 4/10
21/21 [==============================] - 1042s 50s/step - loss: 0.1976 - accuracy: 0.9231 - val_loss: 0.2209 - val_accuracy: 0.8966
Epoch 5/10
21/21 [==============================] - 1033s 49s/step - loss: 0.1881 - accuracy: 0.9276 - val_loss: 0.1885 - val_accuracy: 0.9355
Epoch 6/10
21/21 [==============================] - 1042s 50s/step - loss: 0.1184 - accuracy: 0.9621 - val_loss: 0.1621 - val_accuracy: 0.9325
Epoch 7/10
21/21 [==============================] - 1029s 49s/step - loss: 0.0841 - accuracy: 0.9726 - val_loss: 0.0985 - val_accuracy: 0.9610